In [1]:
# ============================================================
# 0. SETUP
# ============================================================

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')


In [2]:
# ============================================================
# 1. LOAD RAW DATA
# ============================================================

data_path = "car_sales_data.csv"   # change path if needed
df = pd.read_csv(data_path)

print("Shape of raw data:", df.shape)
df.head()


Shape of raw data: (50000, 7)


,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price
0,Ford,Fiesta,1.00,Petrol,2002,127300,3074
1,Porsche,718 Cayman,4.00,Petrol,2016,57850,49704
2,Ford,Mondeo,1.60,Diesel,2014,39190,24072
3,Toyota,RAV4,1.80,Hybrid,1988,210814,1705
4,VW,Polo,1.00,Petrol,2006,127869,4101


In [3]:
# ============================================================
# 2. BASIC SANITY CHECKS
# ============================================================

print("Info:")
df.info()

print("\nMissing values per column:")
print(df.isna().sum())

print("\nNumber of duplicate rows:", df.duplicated().sum())


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         50000 non-null  object 
 1   Model                50000 non-null  object 
 2   Engine size          50000 non-null  float64
 3   Fuel type            50000 non-null  object 
 4   Year of manufacture  50000 non-null  int64  
 5   Mileage              50000 non-null  int64  
 6   Price                50000 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.7+ MB

Missing values per column:
Manufacturer           0
Model                  0
Engine size            0
Fuel type              0
Year of manufacture    0
Mileage                0
Price                  0
dtype: int64

Number of duplicate rows: 12


In [4]:
# ============================================================
# 3. DROP DUPLICATES (IF ANY)
# ============================================================

df = df.drop_duplicates()
print("Shape after removing duplicates:", df.shape)


Shape after removing duplicates: (49988, 7)


In [5]:
# ============================================================
# 4. FEATURE ENGINEERING: AGE & OPTIONAL PRICE_PER_KM
# ============================================================

CURRENT_YEAR = 2025  # adjust if necessary

# Create Age feature
df["Age"] = CURRENT_YEAR - df["Year of manufacture"]

# Optional: price per km (be careful with zero mileage)
df["Price_per_km"] = np.where(df["Mileage"] > 0,
                              df["Price"] / df["Mileage"],
                              np.nan)

print(df[["Year of manufacture", "Age", "Mileage", "Price", "Price_per_km"]].head())
df[["Age", "Price_per_km"]].describe()


   Year of manufacture  Age  Mileage  Price  Price_per_km
0                 2002   23   127300   3074          0.02
1                 2016    9    57850  49704          0.86
2                 2014   11    39190  24072          0.61
3                 1988   37   210814   1705          0.01
4                 2006   19   127869   4101          0.03


,Age,Price_per_km
count,"49,988.00","49,988.00"
mean,20.79,0.49
std,9.65,1.84
min,3.00,0.00
25%,13.00,0.02
50%,21.00,0.08
75%,29.00,0.35
max,41.00,113.99


In [6]:
# ============================================================
# 5. OUTLIER HANDLING (OPTIONAL, IQR METHOD)
# ============================================================
# We will treat extreme outliers in Price and Mileage.
# You can tune the factor if needed (1.5 = standard, 3 = more lenient).

def iqr_filter(data, column, factor=1.5):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - factor * iqr
    upper = q3 + factor * iqr
    filtered = data[(data[column] >= lower) & (data[column] <= upper)]
    return filtered, lower, upper

print("Rows before any outlier removal:", len(df))

# Filter Price
df_price_filtered, lb_price, ub_price = iqr_filter(df, "Price", factor=1.5)
print(f"\nPrice bounds: [{lb_price:.2f}, {ub_price:.2f}]")
print("Rows after Price filtering:", len(df_price_filtered))

# Filter Mileage on top of Price filtering
df_filtered, lb_mileage, ub_mileage = iqr_filter(df_price_filtered, "Mileage", factor=1.5)
print(f"\nMileage bounds: [{lb_mileage:.2f}, {ub_mileage:.2f}]")
print("Rows after Mileage filtering:", len(df_filtered))

# Decide whether to use filtered df or original df
# For now, we proceed with df_filtered as our main working dataframe.
df = df_filtered.copy()
print("\nFinal shape after outlier removal:", df.shape)


Rows before any outlier removal: 49988

Price bounds: [-20893.38, 42981.62]
Rows after Price filtering: 47339

Mileage bounds: [-91840.25, 315213.75]
Rows after Mileage filtering: 46956

Final shape after outlier removal: (46956, 9)


In [7]:
# ============================================================
# 6. HANDLE CATEGORICAL VARIABLES
# ============================================================
# Columns: Manufacturer, Model, Fuel type
# Model can be very high cardinality; often we skip or handle separately.
# Here, we will:
#  - One-hot encode Manufacturer and Fuel type
#  - Drop Model (optional, or keep as raw text for later use)

cat_cols_to_encode = ["Manufacturer", "Fuel type"]
high_cardinality_col = "Model"   # we will drop for modelling to avoid huge dummies

print("\nCardinality before encoding:")
for col in ["Manufacturer", "Model", "Fuel type"]:
    print(f"{col}: {df[col].nunique()} unique values")

# Drop Model for now (you can choose a more advanced encoding later if needed)
df = df.drop(columns=[high_cardinality_col])

# One-hot encoding
df_encoded = pd.get_dummies(df, columns=cat_cols_to_encode, drop_first=True)

print("\nShape after encoding categoricals:", df_encoded.shape)
df_encoded.head()



Cardinality before encoding:
Manufacturer: 5 unique values
Model: 15 unique values
Fuel type: 3 unique values

Shape after encoding categoricals: (46956, 12)


,Engine size,Year of manufacture,Mileage,Price,Age,Price_per_km,Manufacturer_Ford,Manufacturer_Porsche,Manufacturer_Toyota,Manufacturer_VW,Fuel type_Hybrid,Fuel type_Petrol
0,1.00,2002,127300,3074,23,0.02,True,False,False,False,False,True
2,1.60,2014,39190,24072,11,0.61,True,False,False,False,False,False
3,1.80,1988,210814,1705,37,0.01,False,False,True,False,True,False
4,1.00,2006,127869,4101,19,0.03,False,False,False,True,False,True
5,1.40,2018,33603,29204,7,0.87,True,False,False,False,False,True


In [8]:
# ============================================================
# 7. FINAL FEATURE SET FOR MODELLING
# ============================================================
# We keep:
#  - Target: Price
#  - Predictors: Mileage, Engine size, Age, Price_per_km + encoded manufacturers & fuel types

target_col = "Price"

# Drop columns we definitely DO NOT use as predictors
cols_to_drop = ["Year of manufacture"]  # Age already captures this
df_model = df_encoded.drop(columns=cols_to_drop)

print("Columns in df_model:")
print(df_model.columns)

X = df_model.drop(columns=[target_col])
y = df_model[target_col]

print("\nFinal shapes:")
print("X:", X.shape)
print("y:", y.shape)


Columns in df_model:
Index(['Engine size', 'Mileage', 'Price', 'Age', 'Price_per_km',
       'Manufacturer_Ford', 'Manufacturer_Porsche', 'Manufacturer_Toyota',
       'Manufacturer_VW', 'Fuel type_Hybrid', 'Fuel type_Petrol'],
      dtype='object')

Final shapes:
X: (46956, 10)
y: (46956,)


In [9]:
# ============================================================
# 8. SAVE PROCESSED DATA FOR LATER NOTEBOOKS
# ============================================================

# Save full feature-engineered dataframe
df_model.to_csv("car_sales_feature_engineered.csv", index=False)

# Optionally, save X and y separately
X.to_csv("X_features.csv", index=False)
y.to_csv("y_target_price.csv", index=False)

print("Saved:")
print("  - car_sales_feature_engineered.csv")
print("  - X_features.csv")
print("  - y_target_price.csv")


Saved:
  - car_sales_feature_engineered.csv
  - X_features.csv
  - y_target_price.csv


In [10]:
# ============================================================
# 9. QUICK SUMMARY PRINT FOR YOUR THESIS / LOG
# ============================================================

print("=== FEATURE ENGINEERING SUMMARY ===")
print(f"Final dataset shape (after outlier filtering & encoding): {df_model.shape}")
print(f"Number of features (excluding target): {X.shape[1]}")
print("\nSample of columns used as predictors:")
print(X.columns[:20])  # show first 20 columns


=== FEATURE ENGINEERING SUMMARY ===
Final dataset shape (after outlier filtering & encoding): (46956, 11)
Number of features (excluding target): 10

Sample of columns used as predictors:
Index(['Engine size', 'Mileage', 'Age', 'Price_per_km', 'Manufacturer_Ford',
       'Manufacturer_Porsche', 'Manufacturer_Toyota', 'Manufacturer_VW',
       'Fuel type_Hybrid', 'Fuel type_Petrol'],
      dtype='object')
